In [1]:
import numpy as np
import os
from matplotlib import pyplot as plt
import tkinter as tk  
from tkinter import*
from PIL import Image,ImageTk 
from scipy import signal
os.chdir("E:\\Fakultet\\MASTER-DATA SCIENCE\\Image signal processing\\projekat")
slika= plt.imread('lenacolor.tif')

In [2]:
#grupisemo elemente u interval [0,255]
def standard(mat):
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            for k in range(mat.shape[2]):
                if(mat[i,j,k]>255):
                    mat[i,j,k]=255
                if(mat[i,j,k]<0):
                    mat[i,j,k]=0
    return mat

In [3]:
def standardM(mat):
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            if(mat[i,j]>255):
                mat[i,j]=255
            if(mat[i,j]<0):
                mat[i,j]=0
    return mat

In [4]:
def brightness(): 
    nova=standard(slika*float(b1.get())) #mnozimo celu sliku sa faktorom iz intervala [0,2]
    privremena=nova
    nova = Image.fromarray(np.uint8(nova)) #pretvara objekat array koji je input i vraca objekat sliku, np.uint8 opseg 0-255
    nova=ImageTk.PhotoImage(nova) #prebacuje pil sliku u tkinter
    w.create_image(0, 0, image=nova, anchor='nw') #prikaze na canvasu, (0,0) za centriranje, NW-north west za pozicuju slike koje je manja od canvasa
    histogram(np.uint8(privremena))
    master.mainloop()

In [5]:
def saturation(): #mnozimo S kanal nekim faktorom
    image=Image.fromarray(slika).convert('HSV')
    H=image.getchannel('H')
    V=image.getchannel('V')
    
    nova=np.array(image)
    nova=b2.get()*nova
    saturation=standardM(nova.T[1].T) #transponuje pa uzima drugu vrstu zbog S
    satur = Image.fromarray(np.uint8(saturation))

    HSV = Image.merge('HSV', (H, satur, V)) #marge channels, new channel satur
    R,G,B = HSV.convert('RGB').split() #Convert the color from HSV coordinates to RGB coordinates, split da bi se raspakovalo(dimenzije)
    nova = Image.merge('RGB',(R,G,B))
    nova1=ImageTk.PhotoImage(nova)
    w.create_image(0, 0, image=nova1, anchor='nw')
    histogram(np.array(nova))
    master.mainloop()

In [6]:
#izostravanje je preko unsharp maskinga
def sharpness():
    image=Image.fromarray(slika).convert('HSV')
    H=image.getchannel('H')
    S=image.getchannel('S')
    V=image.getchannel('V') 

    h=np.ones((5,5))/25
    V=signal.convolve2d(V,h,'same') 

    newV=np.array(standardM(V),dtype=np.uint8)
    newV=Image.fromarray(newV)
    HSV = Image.merge('HSV', (H, S, newV))
    R,G,B = HSV.convert('RGB').split()
    nova = Image.merge('RGB',(R,G,B))
    nova=np.array(nova)

    c=0.8-float(b3.get()/100)# c je obicno izmedju 0.5-1...c odnos izmedju originalne i zamucene slike
    unsharp=(c/(2*c-1))*slika-((1-c)/(2*c-1))*nova #formula, slika=original, nova=blurred
    unsharp=np.array(standard(unsharp),dtype=np.uint8)
    nova=Image.fromarray(unsharp)
    nova = ImageTk.PhotoImage(nova)
    w.create_image(0, 0, image=nova, anchor='nw')
    histogram(unsharp)
    master.mainloop()

In [7]:
# warmth je odradjen tako da pojacava zutu za toplije i cyan za hladnije boje
def warmth():
    t=b4.get()
    R=slika.T[0]
    G=slika.T[1]
    B=slika.T[2]
    if(t>0):
        R=standardM((1+t)*R) #zuta=zelena+crvena...zelenu i crvenu povecavamo, plavu smanjujemo, jer hocemo toplije boje
        G=standardM((1+t)*G)
        B=standardM((1-t)*B)
    else:
        R=standardM((1+t)*R)# crvena smanjuje, zelena i plava povecava
        G=standardM((1-t)*G)
        B=standardM((1-t)*B)   
    nova=np.zeros((512,512,3),dtype=np.uint8)
    nova.T[0]=R
    nova.T[1]=G
    nova.T[2]=B
    nova1=Image.fromarray(nova)
    nova1=ImageTk.PhotoImage(nova1)
    w.create_image(0, 0, image=nova1, anchor='nw')
    histogram(nova)
    master.mainloop()

In [8]:
#dodaje se na sva tri RGB kanala ista konstanta
def fade():
    t=b5.get()
    nova=np.zeros((512,512,3),dtype=np.uint8)
    R=slika.T[0]+float(t)
    G=slika.T[1]+float(t)
    B=slika.T[2]+float(t)
    nova.T[0]=np.array(standardM(R),dtype=np.uint8)
    nova.T[1]=np.array(standardM(G),dtype=np.uint8)
    nova.T[2]=np.array(standardM(B),dtype=np.uint8)
    nova1=Image.fromarray(nova)
    nova1 = ImageTk.PhotoImage(nova1)
    w.create_image(0, 0, image=nova1, anchor='nw')
    histogram(nova)
    master.mainloop()

In [9]:
#blurujemo celu sliku i onda imamo dve slike original i zamucenu i kombinacijom te dve postizemo zeljeni efekat
def tilt_shift():
    image=Image.fromarray(slika).convert('HSV')
    H=image.getchannel('H')
    S=image.getchannel('S')
    V=image.getchannel('V') 
    Vs=np.array(V)

    h=np.ones((11,11))/121
    V1=signal.convolve2d(V,h,'same')
    newV=np.array(standardM(V1),dtype=np.uint8)

    for i in range(b6.get()):
        Vs[i]=newV[i]
        Vs[511-i]=newV[511-i]

    Vs=Image.fromarray(Vs)
    HSV = Image.merge('HSV', (H, S, Vs))
    R,G,B = HSV.convert('RGB').split()
    nova1 = Image.merge('RGB',(R,G,B))
    nova = ImageTk.PhotoImage(nova1)
    w.create_image(0, 0, image=nova, anchor='nw')
    histogram(np.array(nova1))
    master.mainloop()

In [10]:
# slicno kao gore samo sada zamucujemo sliku van odredjenog kruga centriranog u centru slike
def rad_tilt_shift():
    image=Image.fromarray(slika).convert('HSV')
    H=image.getchannel('H')
    S=image.getchannel('S')
    V=image.getchannel('V') 
    Vs=np.array(V)

    h=np.ones((11,11))/121
    V1=signal.convolve2d(V,h,'same') #noise
    newV=np.array(standardM(V1),dtype=np.uint8) #0-255

    X=np.linspace(0,511,512,dtype=int) #512 number of samples
    maska=np.zeros((512,512))
    for x in X:
        for y in X:
            maska[x,y]=(x-255)**2+(y-255)**2 < (b7.get())**2  #u centru ce nam biti vrednosti manje od w9.get^2...M

    for i in range(512):
        for j in range(512):
            if(maska[i,j]== True):
                newV[i,j]=Vs[i,j]  #kada je ispunjeno M te vrednosti su ustvari originali, oko njih je noise

    newV=Image.fromarray(newV)
    HSV = Image.merge('HSV', (H, S, newV))
    R,G,B = HSV.convert('RGB').split()
    nova = Image.merge('RGB',(R,G,B))
    nova1 = ImageTk.PhotoImage(nova)
    w.create_image(0, 0, image=nova1, anchor='nw')
    histogram(np.array(nova))
    master.mainloop()

In [11]:
def vignette():
    row, col = slika.shape[0], slika.shape[1]
    a= np.ones(row)
    b= np.ones(col)
    slider = b8.get()
    bound_a=(slider/100) *(1/4) * row # mnozimo sa 1/4 zbog maske, na "ivicama" maske ce biti 0=crno, tj. tako smo napravili for petlje
    bound_a = int(bound_a)
    bound_b=(slider/100) * (1/4) * col
    bound_b = int(bound_b)
    for i in range(bound_a):
        a[i] = i / bound_a
        a[row - 1 - i] = a[i] #a je simetrican

    for i in range(bound_b):
        b[i] = i / bound_b
        b[col - 1 - i] = b[i]

    mask=np.zeros((row,col))
    for i in range(row):
        for j in range(col):
            mask[i,j]=a[i]*b[j] #na osnovu 2 for petlje dobijamo masku koja je simetricna matrica sa nulama na ivici i ka centru su vece vrednosti,najveca vrednost je 1

    output = np.copy(slika)
    for i in range(3):
            output[:,:,i] = output[:,:,i] * mask #mnozimo vrednosti sva tri kanala RGB sa maskom, mnozenje sa 1 u centru ostaje ista slika, na rubovima tamnija

    output=output.astype(np.uint8)
    nova = Image.fromarray(output)
    nova=ImageTk.PhotoImage(nova)
    w.create_image(0, 0, image=nova, anchor='nw')
    histogram(output)
    master.mainloop()

In [12]:
#deo po deo linearna transformacija
def contrast():
    t=b9.get()
    nova1=fja(slika,t)
    nova = Image.fromarray(np.uint8(nova1))
    nova=ImageTk.PhotoImage(nova)
    w.create_image(0, 0, image=nova, anchor='nw')
    histogram(np.uint8(nova1))
    master.mainloop()
    
def f(x,A,B):
    if(x<A[0]):
        res = int((A[1]/A[0])*x)
    if(x<=B[0] and x>=A[0]):
        res = int(((B[1]-A[1])/(B[0]-A[0]))*(x-A[0])+A[1])
    if(x>B[0]):
        res = int(((255-B[1])/(255-B[0]))*(x-B[0])+B[1])
    return res
def fja(mat,t):
    A=np.array((85,85-t))
    B=np.array((170,170+t))
    nova=np.zeros((512,512,3),dtype=np.uint8)
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            for k in range(mat.shape[2]):
                x=mat[i,j,k]
                y=f(x,A,B)
                nova[i,j,k]=y
    return nova

Rotation links
[Joel Speranza Math](https://www.youtube.com/watch?v=Z2bR0Mb1Jj0&ab_channel=JoelSperanzaMath)
[Gautam Agrawal](https://gautamnagrawal.medium.com/rotating-image-by-any-angle-shear-transformation-using-only-numpy-d28d16eb5076)

In [13]:
#tehnika 3 shearing
def rotation():
    teta=b10.get()
    teta=np.radians(teta)
    koordinate=[]
    m1=np.array([[1,-np.tan(teta/2)],[0,1]])
    m2=np.array([[1,0],[np.sin(teta),1]])
    m3=np.array([[1,-np.tan(teta/2)],[0,1]])
    for i in range(512): #512x512 dimenzija lene
        for j in range(512):
            kord=np.array([i,j]) #pixele u array
    
            shear1=m1@kord #mnozenje matrica ...dimenzija 2x1..pomera paralelno sa x osom (x,y)-->(x+cy,y)...c=-np.tan(teta/2)
            shear1[0]=int(shear1[0])
            shear1[1]=int(shear1[1])

            shear2=m2@shear1  ##2x1....pomera paralelno sa y osom (x,y)=(x,cx+y)...c=np.sin(teta)
            shear2[0]=int(shear2[0])
            shear2[1]=int(shear2[1])

            shear3=m3@shear2 #..2x1
            shear3[0]=int(shear3[0])
            shear3[1]=int(shear3[1])

            koordinate.append(np.array((shear3),dtype=int))

    pikseli=[]
    for i in range(512):
        for j in range(512):
            pikseli.append(slika[i,j])

    x_mini=np.inf
    x_maks=-np.inf

    y_mini=np.inf 
    y_maks=-np.inf
    for i in range(len(pikseli)):
        if(y_mini>koordinate[i][1]):
            y_mini=koordinate[i][1]
        if(y_maks<koordinate[i][1]):
            y_maks=koordinate[i][1]

    for i in range(len(pikseli)):
        if(x_mini>koordinate[i][0]):
            x_mini=koordinate[i][0]
        if(x_maks<koordinate[i][0]):
            x_maks=koordinate[i][0]

    mx=int(x_maks)- int(x_mini)
    my=int(y_maks)- int(y_mini)
    m=max(mx,my)+1
    zarotirana=np.zeros((m,m,3),dtype=np.uint8)
    for i in range(len(pikseli)):
        x= int(-x_mini) + int(koordinate[i][0])
        y= int(-y_mini) + int(koordinate[i][1])
        zarotirana[x,y]=(pikseli[i])


    poc=int((m-512)/2)
    zarotirana=zarotirana[poc:m-poc, poc:m-poc]
    
    nova = Image.fromarray(np.uint8(zarotirana))
    nova=ImageTk.PhotoImage(nova)
    w.create_image(0, 0, image=nova, anchor='nw')
    histogram(np.uint8(zarotirana))
    master.mainloop()

In [14]:
def NN(): #zoom(nearest neighbors)
    scale = b11.get()
    r = int(scale * slika.shape[0])
    c = int(scale * slika.shape[1])

    zoomed = np.zeros(shape=(r , c , 3) , dtype = 'uint8')

    for i in range(0,r):
        for j in range(0,c):
            zoomed[i,j] = slika[int(i/scale) , int(j/scale)]

    a = int(r // 2)
    h2 = int(a - 256)
    w2 = int(a + 256)        
    zoomed = zoomed[h2:w2 , h2:w2]
    
    nova = Image.fromarray(np.uint8(zoomed))
    nova=ImageTk.PhotoImage(nova)
    w.create_image(0, 0, image=nova, anchor='nw')
    histogram(np.uint8(zoomed))
    master.mainloop()

In [15]:
def refresh():
    nova = Image.fromarray(np.uint8(slika))
    nova = ImageTk.PhotoImage(nova)
    w.create_image(0, 0, image=nova, anchor='nw')
    histogram(slika)
    master.mainloop()

In [16]:
def histogram(slika):
    X=np.linspace(0,255,256,dtype=np.uint8)
    image=Image.fromarray(slika).convert('HSV') #Hue-Saturation-Value...HSV better represents how people relate to colors than the RGB color model does.
    H=image.getchannel('H') #Returns an image containing a single channel of the source image
    S=image.getchannel('S')
    V=image.getchannel('V')
    h=V.histogram()
    
    plt.ioff() #Turn the interactive mode off
    fig = plt.figure()
    plt.bar(X, h)
    plt.grid()
    plt.title("Histogram of image")
    plt.ylim(0,10000)
    plt.savefig('histogram.jpg')
    plt.close(fig)
    
    hist=plt.imread("histogram.jpg")
    hist=ImageTk.PhotoImage(Image.fromarray(hist))
    w.create_image(800, 100, image=hist, anchor='nw')
    master.mainloop()

In [17]:
master = Tk() #standard Python interface to the Tk GUI toolkit
master.geometry("1200x600")

photo=Image.fromarray(np.uint8(slika))
photo=ImageTk.PhotoImage(photo)

w = Canvas(master, width=1200, height=600) #Canvas widget supplies graphics facilities for Tkinter
w.pack()
w.create_image(0, 0, image=photo, anchor='nw')

b1 = Scale(master, from_=0, to=2, orient=HORIZONTAL,resolution=0.1,length=200) #A slider is a Tkinter object with which a user can set a value by moving an indicator
b1.place(x=550,y=0)
Button(master, text="brightness",command=brightness).place(x=630,y=40)#The Button widget is used to add buttons in a Python application

b2=Scale(master, from_=0, to=2, orient=HORIZONTAL,resolution=0.1,length=200)
b2.place(x=550,y=65)
Button(master, text='saturation',command=saturation).place(x=630,y=105)

b3=Scale(master, from_=0, to=+26, orient=HORIZONTAL,resolution=1,length=200)
b3.place(x=550,y=125)
Button(master, text='sharpness',command=sharpness).place(x=630,y=165)

b4=Scale(master, from_=-0.4, to=0.4, orient=HORIZONTAL, resolution=0.1,length=200)
b4.place(x=550,y=185)
Button(master, text='warmth',command=warmth).place(x=630,y=225)

b5=Scale(master, from_=0, to=60, orient=HORIZONTAL, resolution=1,length=200)
b5.place(x=550,y=245)
Button(master, text='fade',command=fade).place(x=630,y=285)

b6=Scale(master, from_=0, to=125, orient=HORIZONTAL, resolution=1,length=200)
b6.place(x=550,y=305)
Button(master, text='tilt shift',command=tilt_shift).place(x=630,y=345)

b7=Scale(master, from_=50, to=200, orient=HORIZONTAL, resolution=1,length=200)
b7.place(x=550,y=365)
Button(master, text='rad tilt shift',command=rad_tilt_shift).place(x=630,y=405)

b8=Scale(master, from_=1, to=100, orient=HORIZONTAL, resolution=1,length=200)
b8.place(x=550,y=425)
Button(master, text='vignette',command=vignette).place(x=630,y=465)

b9=Scale(master, from_=-85, to=85, orient=HORIZONTAL, resolution=1,length=200)
b9.place(x=550,y=485)
Button(master, text='contrast',command=contrast).place(x=630,y=525)

b10=Scale(master, from_=-25, to=25, orient=HORIZONTAL, resolution=1,length=200)
b10.place(x=850,y=485)
Button(master, text='rotation',command=rotation).place(x=900,y=525)

b11=Scale(master, from_=1, to=10, orient=HORIZONTAL, resolution=0.1,length=200)
b11.place(x=850,y=425)
Button(master, text='zoom',command=NN).place(x=900,y=465)

Button(master, text='refresh',command=refresh).place(x=450,y=550)

histogram(slika)
master.mainloop()